In [4]:
import miner_py3_kk as miner
import numpy as np
import pandas as pd
import os
# Path to the miner directory
input_path = os.chdir('/Users/serdarturkaslan/Documents/GitHub/GbmMINER/data/MINER_MicroLowessRNATMM.08.24.2020/')

# create name for results folder where output files will be saved
resultsFolder = "results_miner3_gbm_MicroarrayRNA_survival_08112021"

# name of the folder containing the miner network
#networkFolder = "miner_network_results"
networkFolder = "results_minCorrelation_0o2_50_allFiles_gbm_MicroarrayRNA_survival_08112021"

# create results directory
resultsDirectory = os.path.join(os.getcwd(),resultsFolder)
if not os.path.isdir(resultsDirectory):
    os.mkdir(resultsDirectory)

expressionData=pd.read_csv("GbmMicroRNAMergedWithIDsZScored.csv",header=0,index_col=0,sep=",")
regulonModules = miner.read_json("regulons.json")
overExpressedMembersMatrix = pd.read_csv("overExpressedMembers.csv",index_col=0,header=0)
overExpressedMembersMatrix.index = np.array(overExpressedMembersMatrix.index).astype(str)
underExpressedMembersMatrix = pd.read_csv("underExpressedMembers.csv",index_col=0,header=0)
underExpressedMembersMatrix.index = np.array(underExpressedMembersMatrix.index).astype(str)
eigengenes = pd.read_csv("eigengenes.csv",index_col=0,header=0)
eigengenes.index = np.array(underExpressedMembersMatrix.index).astype(str)
diff_matrix_MMRF = overExpressedMembersMatrix-underExpressedMembersMatrix
# load primary survival data (i.e., corresponding to expression data training set)
survival = pd.read_csv("TCGA_Survival_Gbm.csv",index_col=0,header=0)
survivalDf = survival.iloc[:,0:2]
survivalDf.columns = ["duration","observed"]
kmDf = miner.kmAnalysis(survivalDf=survivalDf,durationCol="duration",statusCol="observed")
guanSurvivalDf= miner.guanRank(kmSurvival=kmDf)
regulonDfMicroGbmLatest = pd.read_csv("regulonDfGbmMicroRNA.csv", header = 0)
regulonDfMicroGbmLatest = list(regulonDfMicroGbmLatest['Regulon_ID'].drop_duplicates())

mgmtpatients = survival[survival['MGMT_Status'] == 'Methylated'].index
nonmgmtpatients = survival[survival['MGMT_Status'] == 'Unmethylated'].index

mesenchymalpatients = survival[survival['Subtype'] == 'mesenchymal'].index
proneuralpatients = survival[survival['Subtype'] == 'proneural'].index
gcimppatients = survival[survival['Subtype'] == 'g-cimp'].index
print("Done")


Done


In [7]:
from lifelines import *
from lifelines.utils import median_survival_times
import matplotlib.pyplot as plt
from lifelines.utils import restricted_mean_survival_time
from lifelines.plotting import rmst_plot
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations


time_limit = 2500
rmst_list = list()
for i in tqdm(regulonDfMicroGbmLatest):
    time.sleep(0.01)
    regulon_num = i
    RegulonActivity = pd.DataFrame(diff_matrix_MMRF.loc[str(regulon_num),:])
    ActiveRegulons = RegulonActivity.index[RegulonActivity[str(regulon_num)]==1]
    InActiveRegulons = RegulonActivity.index[RegulonActivity[str(regulon_num)]==0]
    DownRegRegulons = RegulonActivity.index[RegulonActivity[str(regulon_num)]==-1]
    DownInActiveRegulons = RegulonActivity.index[RegulonActivity[str(regulon_num)]!=1]

    kmf1 = KaplanMeierFitter() 
    srv = guanSurvivalDf.copy()
    patients = list(set(nonmgmtpatients)&set(ActiveRegulons))

    if patients:
        srv1 = srv.loc[patients,["duration"]]
        srv2 = srv.loc[patients,["observed"]]
        ## Fit the data into the model
        kmf_exp = KaplanMeierFitter().fit(list(srv1['duration']), list(srv2['observed']), label='MGMT methylated patients with active (+1) regulon')
        #plt.figure(figsize=(12,12))

        ## Create an estimate
        #ax = plt.subplot(311)

        patients = list(set(nonmgmtpatients)&set(DownRegRegulons))
        if patients:
            srv1 = srv.loc[patients,["duration"]]
            srv2 = srv.loc[patients,["observed"]]

            kmf_con = KaplanMeierFitter().fit(list(srv1['duration']), list(srv2['observed']), label='MGMT methylated patients with down-regulated (-1) regulon')


            #rmst_plot(kmf_exp, model2=kmf_con, t=time_limit, ax=ax)
            #plt.title('Patient survival associated with Regulon-1889')
            #plt.ylabel('Survival probability')
            #plt.xlabel('')  
            #plt.show()
            #print(restricted_mean_survival_time(kmf_con, t=time_limit))
            #print(restricted_mean_survival_time(kmf_exp, t=time_limit))
            rmst_list.append([regulon_num, restricted_mean_survival_time(kmf_con, t=time_limit) - restricted_mean_survival_time(kmf_exp, t=time_limit)])
            #patients = list(set(nonmgmtpatients)&set(DownRegRegulons))
            #srv1 = srv.loc[patients,["duration"]]
            #srv2 = srv.loc[patients,["observed"]]

            #kmf_con1 = KaplanMeierFitter().fit(list(srv1['duration']), list(srv2['observed']), label='Non-MGMT methylated patients with under-active (-1) regulon')

            #ax = plt.subplot(312)
            #plt.title('Patient survival associated with Regulon-1485')
            #rmst_plot(kmf_exp, model2=kmf_con1, t=time_limit, ax=ax)


            #plt.ylabel('Survival probability')
            #plt.xlabel('Overall survival (Days)')  

rmstdf = pd.DataFrame(rmst_list)
rmstdf.columns = ['Regulon_ID', 'RMST_diff_UnderActiveMinusOverActiveMGMTUnMeth']
rmstdf.to_csv("AllRegulonsGBMRMSTValsUnMeth.csv")

  0%|          | 0/3764 [00:00<?, ?it/s]